In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-206718
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-206718


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "ebailey-udacity-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
try:
    cluster = ComputeTarget(ws, cluster_name)
    print("cluster exists already")
except: 
    cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=3)
    cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

cluster.wait_for_completion(show_output=True)


InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.25, 1.75), 
    "max_iter": choice(60, 80, 100, 120, 140)
})

# Specify a Policy
policy = MedianStoppingPolicy()

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(".", "train.py", compute_target=cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name="accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=30, policy=policy, run_config=src)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp.submit(hyperdrive_config)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_693c766d-b133-4986-9aa0-9e28a0dee537,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [22]:
import joblib
# Get your best run and save the model from that run.
runs = list(exp.get_runs())


In [31]:
metrics = runs[0].get_metrics()
parameter_values = runs[0].get_details()['runDefinition']
print(metrics)


{'HD_693c766d-b133-4986-9aa0-9e28a0dee537_25': {'Regularization Strength:': 1.0786362717814155, 'Max iterations:': 120, 'Accuracy': 0.9116843702579667}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_23': {'Regularization Strength:': 1.0177547887019878, 'Max iterations:': 120, 'Accuracy': 0.9119878603945372}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_19': {'Regularization Strength:': 1.067291206340949, 'Max iterations:': 80, 'Accuracy': 0.9106221547799697}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_18': {'Regularization Strength:': 0.35961541131230007, 'Max iterations:': 80, 'Accuracy': 0.903793626707132}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_17': {'Regularization Strength:': 0.3808053003814755, 'Max iterations:': 80, 'Accuracy': 0.9072837632776934}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_8': {'Regularization Strength:': 0.44651103473258086, 'Max iterations:': 120, 'Accuracy': 0.9084977238239758}, 'HD_693c766d-b133-4986-9aa0-9e28a0dee537_13': {'Regularization Strength:': 0.78687459900117

In [34]:
metrics_sorted = sorted(metrics.items(), key=lambda x: x[1]['Accuracy'], reverse=True)
print(metrics_sorted)

[('HD_693c766d-b133-4986-9aa0-9e28a0dee537_6', {'Regularization Strength:': 1.024593044895659, 'Max iterations:': 120, 'Accuracy': 0.9138088012139606}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_4', {'Regularization Strength:': 1.5338825735401653, 'Max iterations:': 60, 'Accuracy': 0.9121396054628225}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_23', {'Regularization Strength:': 1.0177547887019878, 'Max iterations:': 120, 'Accuracy': 0.9119878603945372}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_25', {'Regularization Strength:': 1.0786362717814155, 'Max iterations:': 120, 'Accuracy': 0.9116843702579667}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_15', {'Regularization Strength:': 0.49903842004482246, 'Max iterations:': 80, 'Accuracy': 0.9110773899848255}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_19', {'Regularization Strength:': 1.067291206340949, 'Max iterations:': 80, 'Accuracy': 0.9106221547799697}), ('HD_693c766d-b133-4986-9aa0-9e28a0dee537_24', {'Regularization Strength:': 0.591

In [35]:
best_trial_run = run.get(ws, metrics_sorted[0][0])


Run(Experiment: udacity-project,
Id: HD_693c766d-b133-4986-9aa0-9e28a0dee537_6,
Type: azureml.scriptrun,
Status: Completed)


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###